In [1]:
import sunpy
import sunpy.map
from sunpy.net import vso

import datetime
from datetime import datetime
from datetime import timedelta

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import astropy.units as u
from astropy.io import fits
from astropy.vo.samp import SAMPIntegratedClient

In [36]:
class VSOHandler(object):
    def __init__(self, timestamp, observatory, instrument, detector, measurement):
        self.timestamp = timestamp
        self.askedTime = datetime.strptime(self.timestamp,'%Y/%m/%dT%H:%M:%S.%f')
        
        self.observatory = observatory
        self.instrument = instrument
        self.detector = detector
        self.measurement = measurement
        
        self.__attr = self.__createAttr()
        self.__client = vso.VSOClient()
    
    def __createAttr(self):
        if(self.instrument == 'AIA' or
           self.instrument == 'EIT' or 
           self.instrument == 'SWAP'):
            return self.__createWaveAttr()
        
        elif(self.instrument == 'XRT'):
            #TODO: Filter wheel not supported yet
            return self.__createNoneAttr()
        
        elif(self.instrument == 'HMI'):
            #TODO: how to query HMI
            return self.__createPhysobsAttr()
        
        elif(self.instrument == 'LASCO'):
            return self.__createDetectorAttr()
            
        elif(self.instrument == 'MDI'):
            return self.__createPhysobsAttr()
           
        elif(self.instrument == 'SECCHI' and self.detector == 'EUVI'):
            return self.__createFullAttr()
        
        elif(self.instrument == 'SECCHI'):
            return self.__createSecchiAttr()
        
        elif(self.instrument == 'TRACE'):
            #TODO: trace
            return self.__createWaveAttr()
        
        elif(self.instrument == 'SXT'):
            #TODO: sxt
            return self.__createNoneAttr()
            
            
    def __createWaveAttr(self):
        wave = int(self.measurement)
        return (vso.attrs.Instrument(self.instrument) 
                & vso.attrs.Wave(wave * u.AA, wave * u.AA))
    
    def __createDetectorAttr(self):
        return (vso.attrs.Instrument(self.instrument)
               & vso.attrs.Detector(self.detector))
    
    def __createPhysobsAttr(self):
        obs = self.measurement
        #TODO: 
        if (obs == 'CONTINUUM INTENSITY' or obs == 'FD_Continuum'):
            obs = 'intensity'
        elif (obs == 'MAGNETOGRAMM' or obs == 'FD_Magnetogram'):
            obs = 'LOS_magnetic_field'
        return (vso.attrs.Instrument(self.instrument)
               & vso.attrs.Physobs(obs))
    
    def __createSecchiAttr(self):
        return (vso.attrs.Instrument(self.instrument)
               & vso.attrs.Source(self.observatory)
               & vso.attrs.Detector(self.detector))
    
    def __createFullAttr(self):
        wave = int(self.measurement)
        return (vso.attrs.Instrument(self.instrument)
               & vso.attrs.Source(self.observatory)
               & vso.attrs.Detector(self.detector)
               & vso.attrs.Wave(wave * u.AA, wave * u.AA))
    
    def __createNoneAttr(self):
        return vso.attrs.Instrument(self.instrument)
        
    def createQuery(self, deltaFrom=timedelta(minutes = 2), deltaTo=timedelta(seconds=0)):        
        self.query = self.__client.query(
            vso.attrs.Time(self.askedTime - deltaFrom , self.askedTime + deltaTo),
            self.__attr)
        return self.query
    
    def createQuerySingleResult(self):          
        deltaFromCur = timedelta(minutes = 2)
        deltaFromMin = timedelta(seconds = 0)
        
        # images are usually stored all 3 minutes
        # unless something interesting happens, then it is every 3 seconds
        self.createQuery(deltaFrom = deltaFromCur)
        
        if(len(self.query) > 1):
            self.createQuery(deltaFrom = timedelta(seconds=5))
            if(len(self.query) > 1):
                deltaFromCur = timedelta(seconds=5)
        
        counter = 0
        while(len(self.query) != 1 and counter <= 10):
            
            if(len(self.query) < 1):
                deltaFromMin = deltaFromCur
                deltaFromCur = deltaFromCur * 2
            else:
                deltaFromCur = (deltaFromMin + deltaFromCur)/2
            self.createQuery(deltaFrom = deltaFromCur)
            counter = counter+1
        return self.query
    
    def showQuery(self):
        print(self.query)
    
    def showData(self):
        self.files = self.__client.get(self.query, path='/Data/{instrument}/{file}.fits').wait()
        self.smap = list()
                    
        for file in self.files:
            curMap = sunpy.map.Map(file)
            self.smap.append(curMap)
            plt.figure(figsize=(15,15))

            curMap.plot()
            curMap.draw_limb()

            plt.colorbar()
            plt.show()
        
        
        #self.hdu_list = fits.open(files[0])
        #self.hdu_list.info()
        #self.image_data = hdu_list[0].data
        #self.hdu_list.close()
        #del(self.hdu_list)
        
        #fig = plt.figure(figsize=(20,20))
        #fig.imshow(self.image_data)
        #fig.colorbar()

In [37]:
class Receiver(object):
    def __init__(self, client):
        self.client = client
        self.received = False
    def receive_call(self, private_key, sender_id, msg_id, mtype, params, extra):
        self.params = params
        self.received = True
        self.client.reply(msg_id, {"samp.status": "samp.ok", "samp.result": {}})
    def receive_notification(self, private_key, sender_id, mtype, params, extra):
        self.params = params
        self.received = True
    
    def createHandler(self):
        if self.received:
            timestamp = self.params['timestamp']
            observatory = self.params['observatory']
            instrument = self.params['instrument']
            detector = self.params['detector']
            measurement = self.params['measurement']
            
            self.received = False 
            
            handler = VSOHandler(timestamp, observatory, instrument, detector, measurement)
            
            return handler

In [38]:
client = SAMPIntegratedClient()
client.connect()

r = Receiver(client)
client.bind_receive_call("jhv.vso.load", r.receive_call)
client.bind_receive_notification("jhv.vso.load", r.receive_notification)

In [46]:
r.received

True

In [51]:
# We test every 0.1s to see if the hub has sent a message
import time
while True:
    time.sleep(0.1)
    if r.received:
        handler = r.createHandler()
        break

In [52]:
print('Time: \t\t' + handler.timestamp)
print('Observatory: \t' + handler.observatory)
print('Instrument: \t' + handler.instrument)
print('Detector: \t' + handler.detector)
print('Measurement: \t' + handler.measurement)

Time: 		2017/04/02T12:57:45.350
Observatory: 	SDO
Instrument: 	AIA
Detector: 	
Measurement: 	171


In [53]:
handler.createQuery(timedelta(seconds=10), timedelta(seconds=5))

Start Time [1],End Time [1],Source,Instrument,Type
str19,str19,str3,str3,str8
2017-04-02 12:57:45,2017-04-02 12:57:46,SDO,AIA,FULLDISK


In [49]:
handler.createQuerySingleResult()

Start Time [1],End Time [1],Source,Instrument,Type
str19,str19,str3,str3,str8
2016-01-01 12:49:58,2016-01-01 12:49:59,SDO,AIA,FULLDISK


In [50]:
handler.showData()

In [8]:
handler.smap[0].data

array([[939, 941, 944, ..., 916, 919, 897],
       [932, 933, 940, ..., 909, 912, 899],
       [884, 891, 888, ..., 864, 858, 848],
       ..., 
       [867, 867, 874, ..., 880, 878, 849],
       [865, 868, 869, ..., 884, 878, 849],
       [873, 872, 868, ..., 884, 888, 849]], dtype=int16)